<h1 style="color:pink;">Low Rank Multiplication Rank Included</h1>
<p style = "font-size:20px;">In this notebook, the progress of the Low-Rank Multiplication program is demonstrated. It involves two matrices, where SVD is performed on matrix A while matrix B remains unchanged. The percentage determines the number of vectors to retain from the sigma matrix, which are then used to multiply matrix A by matrix B.</p>

In [5]:
import numpy as np

def LRM_percentage(matrix_A, matrix_B, percentage):
    # SVD decomposition of matrix A
    U, S, VT = np.linalg.svd(matrix_A, full_matrices=False)
    
    # Determine number of singular values to use of the S matrix
    num_singular_values = len(S)
    k = int(np.ceil(num_singular_values * (percentage / 100)))
    k = max(1, min(k, num_singular_values))  # ensure at least 1, at most all

    # Keep only top-k singular values
    S_k = S[:k]
    
    print(f"\nUsing top {k} singular values ({percentage:.1f}%) for LRM")
    print("Original Singular Values (S):", S)
    print("Top-k Singular Values (S_k):", S_k)

    # Step 1: Multiply V^T and B
    step1 = VT[:k, :] @ matrix_B  # only use top-k rows of VT

    # Step 2: Scale each row by Sigma[i,i] and multiply the result of step 1 by the diagonal values of S
    total = np.zeros_like(step1)
    for i in range(k):
        total[i, :] = S_k[i] * step1[i, :]

    # Step 3: Multiply by U
    final_result = U[:, :k] @ total  # only first k columns of U
    print("Final Result of LRM (approx):", final_result, sep="\n")
    
    return final_result

# Example matrices, random numbers between -10 and 10
matrix_A = np.random.randint(-10, 11, size=(3, 4))  
matrix_B = np.random.randint(-10, 11, size=(4, 3))

# Direct multiplication for comparison
direct_result = matrix_A @ matrix_B

# Test LRM with different percentages
percentages = [100, 61.7, 33.3]
for p in percentages:
    result = LRM_percentage(matrix_A, matrix_B, percentage=p)
    error_percentage = (np.linalg.norm(direct_result - result, 'fro') / np.linalg.norm(direct_result, 'fro')) * 100
    print(f"Percentage Error: {error_percentage:.6f}%")



Using top 3 singular values (100.0%) for LRM
Original Singular Values (S): [18.09645109 11.75409107  7.83324972]
Top-k Singular Values (S_k): [18.09645109 11.75409107  7.83324972]
Final Result of LRM (approx):
[[ 46. -57.  81.]
 [-52.  28. -49.]
 [ 56. -58.  41.]]
Percentage Error: 0.000000%

Using top 2 singular values (61.7%) for LRM
Original Singular Values (S): [18.09645109 11.75409107  7.83324972]
Top-k Singular Values (S_k): [18.09645109 11.75409107]
Final Result of LRM (approx):
[[ 46.44703327 -56.15948741  81.6924217 ]
 [-45.94249751  39.38932483 -39.61737352]
 [ 60.27279697 -49.96628105  47.61824869]]
Percentage Error: 12.128907%

Using top 1 singular values (33.3%) for LRM
Original Singular Values (S): [18.09645109 11.75409107  7.83324972]
Top-k Singular Values (S_k): [18.09645109]
Final Result of LRM (approx):
[[-19.49912109  14.06852128 -10.06232502]
 [-27.76716138  20.03387224 -14.32896392]
 [ 41.40529177 -29.87371717  21.36678373]]
Percentage Error: 89.611889%
